In [5]:
import torch
import tensorflow as tf

device=torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(device)
# Optional: Force GPU/CPU usage
if tf.config.list_physical_devices('GPU'):
    device = '/GPU:0'
    print("Using GPU")
else:
    device = '/CPU:0'
    print("Using CPU")

import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load your data
X_train = np.load('/kaggle/input/classifier-data/X_train.npy')
y_train_raw = np.load('/kaggle/input/classifier-data/y_train.npy')
X_test = np.load('/kaggle/input/classifier-data/X_test.npy')
y_test_raw = np.load('/kaggle/input/classifier-data/y_test.npy')

# Convert y to binary labels: 1 for MI, 0 for non-MI
y_train = (y_train_raw.sum(axis=1) > 0).astype(int)
y_test = (y_test_raw.sum(axis=1) > 0).astype(int)

# Feature scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define ANN model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Save the best model based on validation accuracy
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=16,
    callbacks=[early_stop, checkpoint],
    verbose=1
)

# Load the best saved model
best_model = load_model('/kaggle/working/best_model.keras')

# Predict on test set
y_pred_probs = best_model.predict(X_test_scaled)
y_pred = (y_pred_probs > 0.5).astype(int)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


cuda
Using GPU
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1746754625.902267      90 service.cc:148] XLA service 0x7a7538006b50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746754625.902886      90 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1746754626.148689      90 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.3750 - loss: 1.1509

I0000 00:00:1746754627.447732      90 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step - accuracy: 0.4176 - loss: 0.9824
Epoch 1: val_accuracy improved from -inf to 0.69231, saving model to best_model.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 298ms/step - accuracy: 0.4241 - loss: 0.9679 - val_accuracy: 0.6923 - val_loss: 0.6285
Epoch 2/100
1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6250 - loss: 0.6758
Epoch 2: val_accuracy did not improve from 0.69231
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6374 - loss: 0.6555 - val_accuracy: 0.6538 - val_loss: 0.5601
Epoch 3/100
1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6250 - loss: 0.5716
Epoch 3: val_accuracy did not improve from 0.69231
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6809 - loss: 0.5732 - val_accuracy: 0.6538 - val_loss: 0.5424
Epoch 4/100
1/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.9375 - loss: 0.2857
Epoch 4: val_accuracy did not improve from 0.69231
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8763 - loss: 0.3977 - val_accuracy: 0.6154 -